# Problem 54

### Poker Hands

In the card game poker, a hand consists of five cards and are ranked, from lowest to highest, in the following way:

* High Card: Highest value card.
* One Pair: Two cards of the same value.
* Two Pairs: Two different pairs.
* Three of a Kind: Three cards of the same value.
* Straight: All cards are consecutive values.
* Flush: All cards of the same suit.
* Full House: Three of a kind and a pair.
* Four of a Kind: Four cards of the same value.
* Straight Flush: All cards are consecutive values of same suit.
* Royal Flush: Ten, Jack, Queen, King, Ace, in same suit.

The cards are valued in the order:
2, 3, 4, 5, 6, 7, 8, 9, 10, Jack, Queen, King, Ace.

If two players have the same ranked hands then the rank made up of the highest value wins; for example, a pair of eights beats a pair of fives (see example 1 below). But if two ranks tie, for example, both players have a pair of queens, then highest cards in each hand are compared (see example 4 below); if the highest cards tie then the next highest cards are compared, and so on.

The file, poker.txt, contains one-thousand random hands dealt to two players. Each line of the file contains ten cards (separated by a single space): the first five are Player 1's cards and the last five are Player 2's cards. You can assume that all hands are valid (no invalid characters or repeated cards), each player's hand is in no specific order, and in each hand there is a clear winner.

How many hands does Player 1 win?

In [1]:
import collections

Card = collections.namedtuple('Card', 'num suit')

In [2]:
def HighestCard(cards):    
    return True, max(cards, key=lambda c: c.num)

In [3]:
def load_data():
    
    with open('data/p054_poker.txt') as f:
        cards = f.read().split()
        
    i = 0
    while i < len(cards):
        yield cards[i:i+5], cards[i+5:i+10]
        i += 10

In [4]:
dict_card_word_to_number = {k: idx for idx, k in enumerate('23456789TJQKA', start=2)}
print dict_card_word_to_number

def to_program_version(cards):
    for card in cards:
        yield Card(dict_card_word_to_number[card[0]], card[1])

{'A': 14, 'K': 13, 'J': 11, 'Q': 12, '3': 3, '2': 2, '5': 5, '4': 4, '7': 7, '6': 6, '9': 9, '8': 8, 'T': 10}


In [5]:
[(list(to_program_version(hand_player_1)), list(to_program_version(hand_player_2))) 
                                          for hand_player_1, hand_player_2 in load_data()] 

[([Card(num=8, suit='C'),
   Card(num=10, suit='S'),
   Card(num=13, suit='C'),
   Card(num=9, suit='H'),
   Card(num=4, suit='S')],
  [Card(num=7, suit='D'),
   Card(num=2, suit='S'),
   Card(num=5, suit='D'),
   Card(num=3, suit='S'),
   Card(num=14, suit='C')]),
 ([Card(num=5, suit='C'),
   Card(num=14, suit='D'),
   Card(num=5, suit='D'),
   Card(num=14, suit='C'),
   Card(num=9, suit='C')],
  [Card(num=7, suit='C'),
   Card(num=5, suit='H'),
   Card(num=8, suit='D'),
   Card(num=10, suit='D'),
   Card(num=13, suit='S')]),
 ([Card(num=3, suit='H'),
   Card(num=7, suit='H'),
   Card(num=6, suit='S'),
   Card(num=13, suit='C'),
   Card(num=11, suit='S')],
  [Card(num=12, suit='H'),
   Card(num=10, suit='D'),
   Card(num=11, suit='C'),
   Card(num=2, suit='D'),
   Card(num=8, suit='S')]),
 ([Card(num=10, suit='H'),
   Card(num=8, suit='H'),
   Card(num=5, suit='C'),
   Card(num=12, suit='S'),
   Card(num=10, suit='C')],
  [Card(num=9, suit='H'),
   Card(num=4, suit='D'),
   Card(num=1